# 01. Let's import necessary libraries

In [ ]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
import pyemu
import swatp_pst
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatp_pst.__file__

In [ ]:
swatp_pst.__version__

# 00. provide infos

In [ ]:
# working directory for mac
# prj_dir = "/Users/seonggyu.park/Documents/projects/jj_test"
# swatp_wd = "/Users/seonggyu.park/Documents/projects/tools/swatp_pst_wf/models/TxtInOut_Imsil_rye_rot_r1"

In [ ]:
# working directory for win
prj_dir = "D:\\Projects\\Watersheds\\Albufera\\2nd\\opt01"
swatp_wd = "D:\\Projects\\Watersheds\\Albufera\\2nd\\main_opt"

In [ ]:
# calibration period
cal_start = '1/1/2016'
cal_end = '12/31/2023'
# time step
time_step = 'day'
# locations (what our targets)
chs = [49]
stf_obd_file = "cha49_observed.csv"
stf_obd_colnam = "cha049"
irr_obd_file = "irr_paddy_mon.obd.csv"
irr_obd_colnam = "obd_irr_paddy"

# 01. inital setup

In [ ]:
from swatp_pst import handler

In [ ]:
# copy all necessary files (exes) to your working direcotry
handler.init_setup(prj_dir, swatp_wd)

In [ ]:
os.getcwd()

In [ ]:
# create swatp_con file
handler.create_swatp_pst_con(
    prj_dir, swatp_wd, cal_start, cal_end, chs,
    irr_cal=irr_obd_file,
    time_step=time_step)

In [ ]:
os.getcwd()

In [ ]:
wd = os.getcwd()

### Let's do initial run!

In [ ]:
## you need to run the model first if outputs do not exit.
pyemu.os_utils.run(r"swatplus.exe", cwd=".")

# 02. Build template files

In [ ]:
from swatp_pst.pst_utils import PstUtil

In [ ]:
m1 = PstUtil(os.path.join(prj_dir, "main_opt"))

In [ ]:
m1.cal_to_tpl_file()

# 03. Build instruction files

In [ ]:
# extract output first
m1.get_mon_irr()

## 03.01 irrigation

In [ ]:
m1.irr_obd_to_ins("tot_irr_paddy.txt", irr_obd_file, 'obd_irr_paddy', cal_start, cal_end)

## 03.02 streamflow

In [ ]:
m1.extract_day_stf(chs, cal_start, cal_end)

In [ ]:
m1.stf_obd_to_ins('stf_049.txt', stf_obd_file, stf_obd_colnam, cal_start, cal_end)

# 04. Create PEST control file

In [ ]:
os.getcwd()

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)

In [ ]:
par = pst.parameter_data
par

In [ ]:
obd = pst.observation_data
obd

## 05.02 Adjust initial parameter values and their ranges

In [ ]:
# manual

## 05.03 Assign parameter group name

In [ ]:
# set observation group
obd = pst.observation_data
obd

In [ ]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

## 05.04 Provide actual observed values to control file

In [ ]:
# let's do manually

# 06. Create the control file with settings

We can inspect all control data values using the `pst.control_data.formatted_values` attribute. Values are assigned defaults if not specified. Nice.:

In [ ]:
pst.control_data.formatted_values

In [ ]:
pst.control_data.noptmax = 0 # replace 0 with "zero" and see what happens
pst.model_command = 'python forward_run.py'

- ### add new PEST++ variables like so:

In [ ]:
# check the dictionary again
pst.pestpp_options

In [ ]:
pst.write('alb_pst_dummy.pst', version=2)

- ### You can also read and load the existing pest control file.

In [ ]:
# pst_read = pyemu.Pst(os.path.join(swatmf_model,"mb_zon.pst"))

In [ ]:
pst_read.parameter_data